In [1]:
#
#
#    A R I M A   Models
# 
# Auto regressive Integrated Moving Average Models
# Note.. these don't usually work well with stock analysis 
# but they work very well for other things.. and they are 
# widely used
#
# It is a generalization of the ARMA Models

In [3]:
#
# Arima models can be quite complex!
#
# - Seasonal Arima models (more coplex)
# - Non seasonal Arima models (simpler)
#   ARIMA(p,d,q)
#   p : Autoregression
#       A model that utilizes the dependent relationship between a current observation and observations over a previous period
#       (basic regression)
#   d : Integrated
#       Differencing of observations (subtracting an observation from an observation at the previous time step) in order to make 
#       the time series stationary
#   q : Dependency
#       A model that yses tge dependency between an obsevation and a residual error from a moving average model applied to lagged
#       observations